In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
from rosa.preprocessing import (
    clean_cells_genes,
)

RAW_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features.h5ad"
EMBEDS_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"


In [ ]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)

In [ ]:
adata.layers['counts'].sum(axis=1)

In [ ]:
adata.layers['counts_normalized'] = adata.layers['counts'].copy()
sc.pp.normalize_total(adata, target_sum=1e5, layer='counts_normalized')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(adata.layers['counts_normalized'].flatten());

In [ ]:
from rosa.preprocessing import bin_expression, reconstruct_expression

In [ ]:
bin_expression(adata, 128)

In [ ]:
reconstruct_expression(adata)

In [ ]:
((adata.X - adata.layers['reconstructed'])**2).mean()

In [ ]:
# Plot residuals
plt.hist((adata.layers['reconstructed'] - adata.X).ravel(), bins=1000);
plt.xlim([-.25, .25]);

In [ ]:
# Identify cells and genes not trained on (when possible)
adata.layers['prediction'] = adata.layers['reconstructed']
test_genes = np.logical_not(adata.var["train"])
test_cells = np.logical_not(adata.obs["train"])
adata_test = adata[test_cells, test_genes]
sc.tl.dendrogram(adata_test, groupby="label", use_rep="X")


In [ ]:
from rosa.plotting import plot_marker_gene_heatmap

In [ ]:
marker_genes_dict = adata_test.obs.set_index('label').to_dict()['marker_feature_name']
plot_marker_gene_heatmap(adata_test, marker_genes_dict)

In [ ]:
plt.hist(adata.layers['binned'].flatten(), bins=25, density=True);

In [ ]:
plt.hist(adata.X.flatten(), bins=250, density=True);
plt.ylim([0, 1]);
plt.xlim([0, 8])

In [ ]:
adata.X

In [ ]:
from typing import Union

In [ ]:
adata.layers['X']

In [ ]:
np.empty((10, 0))[0]

In [ ]:
from enum import Enum, auto

class EmbeddingType(Enum):
    JOINT = auto()
    VAR = auto()
    OBS = auto()

In [ ]:
EmbeddingType.JOINT

In [ ]:
list(EmbeddingType.__members__)

In [ ]:
adata.X = np.ceil(adata.X)
sc.pp.filter_genes(adata, min_cells=1)
sc.experimental.pp.normalize_pearson_residuals(adata)
adata.X[adata.X<0] = 0

In [ ]:
2**10

In [ ]:
import torch

In [ ]:
np.isinf(adata.X).sum()

In [ ]:
np.isnan(adata.X).sum()

In [ ]:
sc.pp.log1p(adata)

In [ ]:
adata.X

In [ ]:
torch.tensor([0])

In [ ]:
adata.X = np.ceil(adata.X)
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.filter_genes(adata, min_cells=1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=1)

In [ ]:
adata

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(adata.layers['counts'].sum(axis=1));

In [ ]:
print(adata.layers['counts'].sum(axis=1).mean() / 1e5)
print(adata.layers['counts'].sum(axis=1).var() / 1e10)

In [ ]:
adata.layers["counts_normalized_total"] = adata.X.copy()
sc.pp.normalize_total(adata, 1e5, layer="counts_normalized_total")

In [ ]:
print(adata.layers['counts_normalized_total'].sum(axis=1).mean() / 1e5)
print(adata.layers['counts_normalized_total'].sum(axis=1).var() / 1e10)

In [ ]:
adata.layers["counts_normalized_pearson"] = adata.X.copy()
adata.layers['counts_normalized_pearson'] = np.ceil(adata.layers['counts_normalized_pearson'])
sc.experimental.pp.normalize_pearson_residuals(adata, layer="counts_normalized_pearson", theta=1e2)

In [ ]:
np.isnan(adata.layers['counts_normalized_pearson']).sum()

In [ ]:
print(adata.layers['counts_normalized_pearson'].sum(axis=1).mean())
print(adata.layers['counts_normalized_pearson'].sum(axis=1).var())

In [ ]:
plt.hist(adata.layers['counts_normalized_pearson'].flatten(), np.linspace(0, 100, 1000));

In [ ]:
plt.hist(adata.layers['counts'].flatten(), np.linspace(0, 100, 1000));

In [ ]:
np.log1p(0.1)

In [ ]:
(adata.layers['counts_normalized_pearson'] - adata.layers['counts']).max()

In [ ]:
adata.layers['counts_normalized_pearson'] = np.round(adata.layers['counts_normalized_pearson'])

In [ ]:
np.isnan(adata.layers['counts_normalized_pearson'])[0]

In [ ]:
adata.layers['counts_normalized_pearson'].min()

In [ ]:
adata.uns['pearson_residuals_normalization']

In [ ]:
TABULA_SAPIENS_BY_CELL_TYPE_WITH_EMBEDS_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"

In [ ]:
adata_norm = ad.read_h5ad(TABULA_SAPIENS_BY_CELL_TYPE_WITH_EMBEDS_PT)

In [ ]:
y = np.exp(adata_norm.X) - 1

In [ ]:
y

In [ ]:
import scanpy as sc

In [ ]:
adata_norm.X = adata_norm.layers['counts'].copy()

In [ ]:
sc.pp.normalize_total(adata_norm, 1e5)

In [ ]:
(abs(adata_norm.X - y)).max()

In [ ]:
# from scipy.special import kl_div

# y_hat = np.asarray(adata[keep_cells].X.flatten())
# y = np.asarray(adata[keep_cells].layers['prediction'].flatten())

# kl_div(y, y_hat).mean()

In [ ]:
from scipy.stats import kstest, poisson

y_hat = np.asarray(adata[keep_cells].X.flatten())
y = np.asarray(adata[keep_cells].layers['prediction'].flatten())

kstest(y_hat, y)

In [ ]:
kstest(y, 'poisson', args=(np.mean(y),))

In [ ]:
from scipy.stats import kstest, poisson

poisson_dist = poisson(np.mean(y))

In [ ]:
y_p = poisson_dist.rvs(size=10000)

In [ ]:
hist, _ = np.histogram(y, bins=bins)
hist_hat, _ = np.histogram(y_hat, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_hat/hist_hat.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
poisson_dist = poisson(np.mean(y))
y_new = poisson_dist.rvs(size=10000)


hist, _ = np.histogram(y, bins=bins)
hist_new, _ = np.histogram(y_new, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_new//hist_new.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
initial_params = [.2, 0.1, 1.2, 0.0.001]

result = minimize(negative_binomial, initial_params, args=(y,), method='Nelder-Mead')

In [ ]:
# Extract the optimal parameters
r1, p1, r2, p2 = result.x
data = y

In [ ]:
from scipy.stats import nbinom

r1, p1, r2, p2 = (0.1, 0.1, 1.1, 0.1)

# nbinom_dist_1 = nbinom(9.1, 0.6)
# nbinom_dist_2 = nbinom(1.2, .001)
y_new = (nbinom.rvs(.2, 0.1, size=10000) + nbinom.rvs(1.2, 0.001, size=10000)) / 1000


hist, _ = np.histogram(y, bins=bins)
hist_new, _ = np.histogram(y_new, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_new/hist_new.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import nbinom
import matplotlib.pyplot as plt

# Generate sample data
data = y

# Define the negative binomial function
def negative_binomial(params, data):
    r1, p1, r2, p2 = params
    pmf1 = nbinom.pmf(1000 * data, r1, p1)
    pmf2 = nbinom.pmf(1000 * data, r2, p2)
    return -np.log(pmf1 + pmf2).sum()

# Define the initial values for the parameters
initial_params = (0.1, 0.1, 1.1, 0.1)


# Minimize the negative binomial function using the Nelder-Mead algorithm
result = minimize(negative_binomial, initial_params, args=(data,), method='Nelder-Mead')

# Extract the optimal parameters
r1, p1, r2, p2 = result.x

# Plot the histogram of the data
plt.hist(data, bins=30, density=True, alpha=0.5, label='Data')

# Plot the sum of the negative binomials
x = np.arange(0, data.max())
pmf1 = nbinom.pmf(x, r1, p1)
pmf2 = nbinom.pmf(x, r2, p2)
plt.plot(x, pmf1 + pmf2, 'r-', lw=2, label='Sum of Negative Binomials')

plt.xlabel('x')
plt.ylabel('Probability')
plt.legend()
plt.show()


In [ ]:
try:
  print(x)
except NameError:
  print("Variable x is not defined")

In [ ]:
import torch

In [ ]:
torch.long

In [ ]:
a = torch.tensor(2.3).type(torch.long)

In [ ]:
adata.varm['embedding'].shape

In [ ]:
import matplotlib.pyplot as plt

plt.hist(adata.varm['embedding'].ravel(), bins=2000);
plt.xlim([-0.5, 0.5]);

In [ ]:
adata.varm['embedding']

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# fit pca on training data
pca = PCA()
pca.fit(adata.varm['embedding'])


In [ ]:
E = pca.transform(adata.varm['embedding'])

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_));
plt.xlim([0, 3042])

In [ ]:
np.cumsum(pca.explained_variance_ratio_)[512]

In [ ]:
plt.hist(E[:, :512].ravel(), bins=2000);
plt.xlim([-1.5, 1.5]);

In [ ]:
E.shape

In [ ]:
import anndata as ad
from rosa.preprocessing import (
    calculate_gene_embeddings_pca,
)


EMBEDS_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"

In [ ]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)
adata = calculate_gene_embeddings_pca(adata, 256)

In [ ]:
adata.uns["embedding_pca"]

In [ ]:
adata.write_h5ad(EMBEDS_ADATA_PT)

In [ ]:
adata.varm['embedding_pca'].shape

In [ ]:
adata.varm

In [ ]:
ADATA_BULK_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_pbulk.h5ad"
adata = ad.read_h5ad(ADATA_BULK_PT)


In [ ]:
adata.var.set_index("feature_id")

In [ ]:
base_pt = '/home/ec2-user/enformer/Homo_sapiens.GRCh38.genes.enformer_embeddings'
var_id = 'ENSG00000280445'
full_pt = f'{base_pt}/{var_id}.pt'

In [ ]:
import torch

In [ ]:
for i in range(64):
    var = torch.load(full_pt, map_location='cpu')['embedding']
    var = torch.from_numpy(var).type(torch.float32)

In [ ]:
type(var)

In [ ]:
896 // 2

In [ ]:
a = var.unsqueeze(0).unsqueeze(-3)
a.shape

In [ ]:
fc = torch.nn.Conv2d(1, 10, (896, 1))

In [ ]:
fc(a).view(a.shape[0], -1).shape

In [ ]:
view(a.shape[0], -1)

In [ ]:
import torch.nn.functional as F


class ToTensor(torch.nn.Module):
    """Convert ``numpy.ndarray`` to tensor.
    """
    def __init__(self, dtype: torch.dtype = torch.float32) -> None:
        super().__init__()
        self.dtype = dtype

    def forward(self, tensor: np.ndarray) -> torch.Tensor:
        return torch.from_numpy(tensor).type(torch.float32)

class CountNormalize(torch.nn.Module):
    """Normalize a tensor to a fixed total counts.
    """
    def __init__(self, total_counts=1):
        super().__init__()
        self.total_counts = total_counts

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        return self.total_counts * F.normalize(tensor, p=1.0, eps=1e-12)


class Log1p(torch.nn.Module):
    """Log1p normalize a tensor.
    """
    def __init__(self):
        super().__init__()

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        return torch.log1p(tensor)


class QuantileNormalize(torch.nn.Module):
    """Normalize a tensor by quantiles.
    """
    def __init__(self, n_bins):
        super().__init__()
        self.n_bins = n_bins

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        boundaries = torch.quantile(tensor, torch.linspace(0, 1, self.n_bins))
        return torch.bucketize(tensor, boundaries)

In [ ]:
class ExpressionTransform(torch.nn.Sequential):
    def __init__(self, cfg):
        # Add base transform
        transforms = [ToTensor()]

        if cfg.total_counts is not None:
            transforms.append(CountNormalize(cfg.total_counts))

        if cfg.log1p:
            transforms.append(Log1p())

        if cfg.n_bins is not None:
            transforms.append(QuantileNormalize(cfg.n_bins))

        super().__init__(*transforms)


from dataclasses import dataclass
from typing import Optional


@dataclass
class ExpressionTransformConfig:
    total_counts: Optional[int] = None
    log1p: Optional[bool] = None
    n_bins: Optional[int] = None

In [ ]:
exp_cfg = ExpressionTransformConfig(n_bins=10)

In [ ]:
tf = ExpressionTransform(exp_cfg)

In [ ]:
torch.quantile(ToTensor()(X).unsqueeze(0), torch.linspace(0, 1, 5), dim=-1, keepdim=True).shape

In [ ]:
ToTensor()(X).unsqueeze(0).shape

In [ ]:
tf(np.random.rand(20))

In [ ]:
X[1]

In [ ]:
a = torch.nn.Sequential(ToTensor(), CountNormalize(10), Log1p(), QuantileNormalize(10))
b = torch.nn.Sequential(ToTensor(), QuantileNormalize(10))

In [ ]:
X = np.random.rand(3, 30)
X [0, :3] = 0
X [0, :] = X [0, :] * 1000
# X = torch.randint(1, 5, size=(3, 3))

In [ ]:
X

In [ ]:
a(X)

In [ ]:
b(X)

In [ ]:
a(X).sum(dim=1)

In [ ]:
20 * 512 / 1e3

In [ ]:
type(torch.float32)

In [ ]:
import numpy as np

In [ ]:
np.unravel_index(10, (9, 2))

In [ ]:
EMBEDS_ADATA_PT = '/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad'

In [ ]:
EMBEDS_ADATA_PT_2 = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_raw.h5ad"


In [ ]:
import anndata as ad

In [ ]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)
adata_2 = ad.read_h5ad(EMBEDS_ADATA_PT_2)

In [ ]:
adata.layers['counts'][0, 0]

In [ ]:
adata_2.X.sum(axis=1).mean()

In [ ]:
adata.X[0, 0]

In [ ]:
adata.layers['binned'][0, 0]

In [ ]:
adata_2.X[0, 0]

In [ ]:
from enum import Enum, auto
from typing import Optional


class ExpressionActivations(Enum):
    SOFTPLUS = auto()
    SOFTMAX = auto()

In [ ]:
str(ExpressionActivations.SOFTPLUS)

In [ ]:
ExpressionActivations.SOFTPLUS.name.lower()

In [ ]:
import torch.nn as nn

In [ ]:
a=nn.Sequential()

In [ ]:
a.append(nn.Linear(1, 1))

In [ ]:
body = nn.Identity()

In [ ]:
body(torch.rand(10, 10)).shape

In [ ]:
map((body, body), (torch.rand(10, 10), torch.rand(10, 10)))

In [ ]:
torch.add((torch.rand(10, 10), torch.rand(10, 10))).shape

In [ ]:
torch.cat((torch.rand(10, 10), torch.rand(10, 10)), dim=-1).shape

In [ ]:
from typing import Tuple

class AttentionEmbeds(nn.Module):
    def __init__(self, in_dim: Tuple[int, int], out_dim) -> None:
        super(AttentionEmbeds, self).__init__()
        
        self.value = nn.Parameter(torch.randn(out_dim))
        self.activation = nn.GELU()
        self.out_dim = out_dim

    def forward(self, x: Tuple[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        atten = self.activation(torch.einsum('...i, ...i ->...', *x))
        return torch.einsum('..., i -> ...i', atten, self.value)

In [ ]:
x_1 = torch.rand((10, 20))
x_2 = torch.rand((10, 20))
v = torch.rand(20)

In [ ]:
atten = torch.einsum('...i, ...i ->...', x_1, x_2)

In [ ]:
out = torch.einsum('..., i -> ...i', atten, v)

In [ ]:
nn.Parameter(torch.randn(20))

In [ ]:
a = AttentionEmbeds((20, 20), 30)

In [ ]:
a((x_1, x_2)).shape

In [ ]:
X = np.random.rand(100, 896, 3072)

In [ ]:
PT = '/Users/nsofroniew/Documents/data/multiomics/enformer/scratch'

In [ ]:
import zarr

In [ ]:
z1 = zarr.open(PT + '/example.zarr', mode='w', shape=(1000, 896, 3072), chunks=(1, None, None), dtype='float32')

In [ ]:
for i in range(10):
    z1[i * 100: (i+1)*100] = X

In [ ]:
z2 = zarr.open(PT + '/example.zarr', mode='r')

In [ ]:
from time import time

In [ ]:
start = time()
z2[893]
stop = time()
print(stop - start)

In [ ]:
torch.save({'results': X[0]}, PT + '/example_0.pt')

In [ ]:
start = time()
torch.load(PT + '/example_0.pt')
stop = time()
print(stop - start)

In [ ]:
class ZarrDataset(torch.utils.data.Dataset):
    def __init__(self, path: str):
        super(ZarrDataset, self).__init__()

        self.path = path

        self.array = zarr.open(path, mode='r')

    def __len__(self) -> int:
        return self.array.shape[0]

    def __getitem__(self, idx):
        return self.array[idx]

In [ ]:
ds = ZarrDataset(PT + '/example.zarr')
dl = torch.utils.data.DataLoader(ds, shuffle=False, num_workers=2)

In [ ]:
start = time()
for batch in iter(dl):
    pass
stop = time()
print(stop - start)

In [1]:
from rosa.datasets import RosaObsDataset, ToTensor, RosaObsVarDataset, RosaJointDataset
from rosa.config import ExpressionTransformConfig

from torch.utils.data import default_collate


Global seed set to 0
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [ ]:
# isinstance(ds, RosaJointDataset)

In [ ]:
import anndata as ad

ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"


adata = ad.read_h5ad(ADATA_PT)

In [ ]:
from torch import Tensor
from typing import Optional, Tuple, List

In [ ]:
ds = RosaObsDataset(adata, obs_input='embedding')

In [ ]:
ds[0][0].shape

In [ ]:
ds[0][1].shape

In [ ]:
adata.varm['embedding_pca'].shape

In [ ]:
2**14 / 19429

In [ ]:
import torch

torch.empty((0, 0))

In [ ]:
# class RosaObsVarDataset(RosaJointDataset):
#     def __init__(
#         self,
#         adata: ad.AnnData,
#         *,
#         var_input: str,
#         obs_input: str,
#         expression_layer: Optional[str] = None,
#         expression_transform_config: Optional[ExpressionTransformConfig] = None,
#     ) -> None:
#         super(RosaObsVarDataset, self).__init__(adata, obs_input=obs_input, var_input=var_input, expression_layer=expression_layer, expression_transform_config=expression_transform_config)

#     def __len__(self) -> int:
#         return self.expression.shape[0]

#     def __getitem__(self, idx: int) -> Tuple[Tuple[Tensor, Tensor], Tensor]:
#         obs_input = self.input[0][idx]
#         expression = self.expression[idx]
#         full_input = (obs_input.expand((self.input[1].shape[0], obs_input.shape[0])), torch.empty((self.input[1].shape[0], 0)))
#         return full_input, expression

#     def collate_fn(self, batch: List[Tuple[Tuple[Tensor, Tensor], Tensor]]) -> Tuple[Tuple[Tensor, Tensor], Tensor]:
#         (x0, _), y = default_collate(batch)
#         x1 = self.input[1].expand((x0.shape[0],) + self.input[1].shape)
#         return (x0.view(-1, x0.shape[-1]), x1.view(-1, x1.shape[-1])), y.view(-1)

In [ ]:
ds = RosaObsVarDataset(adata, obs_input='embedding', var_input='embedding_pca')

In [ ]:
ds[0][0][0].shape

In [ ]:
ds[0][0][1].shape

In [ ]:
ds[0][1].shape

In [ ]:
from torch.utils.data import DataLoader


dl = DataLoader(
            ds,
            batch_size=64,
            shuffle=False,
            num_workers=0,
            # collate_fn=ds.collate_fn,
        )

In [ ]:
batch = next(iter(dl))

In [ ]:
batch[0][0].shape

In [ ]:
batch[0][1][0].expand(batch[0][1].shape).shape

In [ ]:
batch[1].shape

In [ ]:
from torch.utils.data import default_collate


def obsvar_collate(batch):
    (x0, x1), y = default_collate(batch)
    return (x0.view(-1, x0.shape[-1]), x1.view(-1, x1.shape[-1])), y.view(-1)

In [ ]:
x = ds[0][0][0]

In [ ]:
x.expand((2000, 110)).shape

In [2]:
import torch

torch.arange(100)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [11]:
z = torch.multinomial(torch.arange(100).float(), 10).long()

In [9]:
x = torch.rand(100, 1000)

In [13]:
x[z].shape

torch.Size([10, 1000])